In [1]:
import pandas as pd
import geopandas as gpd
import json
from pandas import json_normalize
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
from requests.structures import CaseInsensitiveDict


GPEtags = pd.read_csv("20species_GPEtags.csv", sep = ";")

In [2]:
GPEtags['Countries'] = GPEtags['Countries'].str.replace("Vatican", "Vatican City")

In [3]:
locs = []

for txt in GPEtags['Countries']:
    text = txt
    url = "https://api.geoapify.com/v1/geocode/search?text=" + text + "&limit=1&apiKey=3e1f56207e074f7a8108362ba0a7e5c5"
    
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"

    resp = requests.get(url, headers=headers)
    locs.append(resp.json()) 

In [4]:
test = []

for n in range(len(locs)):

    geom = {'geometry' : (locs[n]['features'][0]['properties']['lon'], locs[n]['features'][0]['properties']['lat'])}
    name = {'name' : locs[n]['features'][0]['properties']['country']}
    
    for g, n in zip(geom, name):
        t = (name[n], geom[g])
        test.append(t)

In [5]:
geom = pd.DataFrame(test, columns =['Countries_x', 'geometry'])

In [6]:
GPEtags = pd.concat([GPEtags, geom], axis = 1)

In [7]:
from shapely.geometry import Point

GPEtags['Coordinates'] = GPEtags['geometry'].apply(Point)

gdf = gpd.GeoDataFrame(GPEtags, geometry="Coordinates")

In [8]:
gdf

,scientific_name,Countries,Country_count,article_count,Countries_x,geometry,Coordinates
0,Acinonyx jubatus,India,43,50,India,"(78.6677428, 22.3511148)",POINT (78.66774 22.35111)
1,Acinonyx jubatus,Iran,35,50,Iran,"(54.5643516, 32.6475314)",POINT (54.56435 32.64753)
2,Acinonyx jubatus,Namibia,5,50,Namibia,"(17.3231107, -23.2335499)",POINT (17.32311 -23.23355)
3,Acinonyx jubatus,South Africa,5,50,South Africa,"(24.991639, -28.8166236)",POINT (24.99164 -28.81662)
4,Acinonyx jubatus,China,4,50,China,"(104.999927, 35.000074)",POINT (104.99993 35.00007)
...,...,...,...,...,...,...,...
652,Ursus thibetanus,Singapore,1,95,Singapore,"(103.8194992, 1.357107)",POINT (103.81950 1.35711)
653,Ursus thibetanus,South Africa,1,95,South Africa,"(24.991639, -28.8166236)",POINT (24.99164 -28.81662)
654,Ursus thibetanus,Spain,1,95,Spain,"(-4.8379791, 39.3260685)",POINT (-4.83798 39.32607)
655,Ursus thibetanus,Sri Lanka,1,95,Sri Lanka,"(80.7137847, 7.5554942)",POINT (80.71378 7.55549)


In [19]:
# Create a world map to show distributions of users 
# NOT A GOOD VISUALIZATION need to demonstrate size according to species count value
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
#empty map
world_map= folium.Map(tiles="cartodbpositron")
#m = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(gdf)):
    lat = gdf.iloc[i]['Coordinates'].y
    long = gdf.iloc[i]['Coordinates'].x
    radius=float(gdf.iloc[i]['Country_count'])*4000
    popup_text = """Country : {}<br>
                Species : {}<br>"""
    popup_text = popup_text.format(gdf.iloc[i]['Countries'],
                                gdf.iloc[i]['scientific_name'])
    folium.Circle(location = [lat, long], radius = radius, popup= popup_text, fill =True).add_to(world_map)
#show the map
world_map